In [133]:
using Interact
using LCMGL
using ForwardDiff
using RigidBodyDynamics
using DataFrames
using SimpleGradientDescent
using Ipopt

In [113]:
reload("SpatialFields")

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/lib/v0.5/SpatialFields.ji for module SpatialFields.


In [114]:
include("../src/arms.jl")
include("../src/depth_sensors.jl")
include("../src/depthdata.jl")

DepthData

In [115]:
squished = DepthData.read_point_cloud(open("data/squishable_squished_xyzrgb.txt"))

PointCloud with origin: [1.37689,0.775602,1.4848] containing 25164 points

In [116]:
DepthData.render_lcmgl(squished)

In [117]:
unsquished = DepthData.read_point_cloud(open("data/squishable_unsquished_xyzrgb.txt"))

PointCloud with origin: [1.38246,0.768824,1.48581] containing 25571 points

In [118]:
DepthData.render_lcmgl(unsquished)

In [119]:
model = Arms.squishable()
state = Arms.ManipulatorState(model)
Arms.draw(state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 294xFixedSizeArrays.Point{3,Float64},     faces: 584xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f65f4012b40>))

In [120]:
function flatten(state::Arms.ManipulatorState)
    vcat(state.mechanism_state.q, state.deformation_data)
end

function unflatten!{T}(state::Arms.ManipulatorState, data::AbstractVector{T})
    nq = num_positions(state.mechanism_state)
    set_configuration!(state.mechanism_state, data[1:nq])
    state.deformation_data[:] = data[(nq + 1):end]
end

unflatten! (generic function with 7 methods)

In [128]:
point_cloud = unsquished
DepthData.render_lcmgl(point_cloud)
sensed_points = [p.position for p in point_cloud.points]

cost = x -> begin
    typealias T eltype(x)
    state = Arms.ManipulatorState(model, T, T)
    unflatten!(state, x)
    Arms.draw(state, false)
    skin = Arms.skin(state)[1]
    raycast_cost = zero(T)
    for point in sensed_points
        raycast_cost += skin(point)^2
    end
#     raycast_cost = sum(point -> skin(point)^2, sensed_points)
    deformation_cost = zero(T)
    for ld in values(state.deformations)
        for d in ld
            deformation_cost += sum(d .^2)
        end
    end
    return raycast_cost / length(sensed_points) + 100*deformation_cost
end

# Use autodiff to prepare a function that will evaluate the gradient
# of that cost:
cost_and_gradient = x -> begin
    out = GradientResult(x)
    ForwardDiff.gradient!(out, cost, x)
    ForwardDiff.value(out), ForwardDiff.gradient(out)
end

trials = []


function test_gradient_descent()
    for i = 1:1
        xs = []

        # We'll initialize our estimate with a uniformly random pair
        # of joint angles
        x_estimated = flatten(Arms.ManipulatorState(model))
        x_estimated[5:7] = [.5, 0, .5]

        # Now we use gradient descent to recover the joint angles
        for i = 1:50
            push!(xs, x_estimated)

            # Evaluate the gradient of the point values w.r.t. the
            # joint angles:
            ci, gi = cost_and_gradient(x_estimated)

            if any(isnan, gi)
                @show x_estimated
                error("NaN in gradient")
            end
            
            # Descend the gradient
            step = -ci ./ gi
            step[1:7] = sign(step[1:7]) .* min.(0.01, abs.(gi[1:7]))
            step[8:end] = sign(step[8:end]) .* min.(1e-5, abs.(gi[8:end]))
            x_estimated += step

            # Descend the gradient
#             x_estimated -= 0.0000002 * gi
            
            # TODO: generalize this
            x_estimated[1:4] = x_estimated[1:4] / norm(x_estimated[1:4])
        end
        push!(trials, DataFrame(x=xs))
    end
end

test_gradient_descent (generic function with 1 method)

In [129]:
test_gradient_descent()
@time test_gradient_descent()
state = Arms.ManipulatorState(model)
unflatten!(state, trials[1][:x][end])
Arms.draw(state)

  5.270178 seconds (64.10 M allocations: 4.641 GB, 14.09% gc time)


DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 310xFixedSizeArrays.Point{3,Float64},     faces: 616xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f65f4012bd0>))

In [136]:
x0 = flatten(Arms.ManipulatorState(model))
num_vars = length(x0)
solver = IpoptSolver(print_level=1, max_iter=100)
m = getmodel(GradientDescent(cost), num_vars, solver)
setwarmstart!(m, x0);

In [137]:
optimize!(m)
x = getsolution(m)
state = Arms.ManipulatorState(model)
unflatten!(state, x)
Arms.draw(state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 306xFixedSizeArrays.Point{3,Float64},     faces: 608xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f65f4012c30>))